# 📌 PhonePe Transaction Insights - Full Colab Notebook

In [ ]:
# 1️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2️⃣ Clone PhonePe Pulse GitHub repo
!git clone https://github.com/PhonePe/pulse.git
!ls pulse/data

In [ ]:
# 3️⃣ Install required packages
!pip install pandas plotly matplotlib ipywidgets

In [ ]:
# 4️⃣ Import libraries
import os
import json
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact, widgets

In [ ]:
# 5️⃣ Setup SQLite database and load data
conn = sqlite3.connect("phonepe_pulse.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS aggregated_transaction (
    state TEXT,
    year INTEGER,
    quarter INTEGER,
    name TEXT,
    count INTEGER,
    amount REAL
)
""")

# Load JSON data into SQLite
base_path = "pulse/data/aggregated/transaction/country/india/state/"

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if file.endswith(".json"):
                with open(os.path.join(year_path, file)) as f:
                    data = json.load(f)
                    if data["data"]["transactionData"]:
                        for txn in data["data"]["transactionData"]:
                            cursor.execute("""
                                INSERT INTO aggregated_transaction
                                (state, year, quarter, name, count, amount)
                                VALUES (?, ?, ?, ?, ?, ?)
                            """, (
                                state.replace("-", " ").title(),
                                int(year),
                                int(file.strip(".json")),
                                txn["name"],
                                txn["paymentInstruments"][0]["count"],
                                txn["paymentInstruments"][0]["amount"]
                            ))

conn.commit()
print("✅ Data loaded successfully into SQLite DB")

In [ ]:
# 5️⃣b Load District-Level Transaction Data into SQLite
import os, json, sqlite3

# Create the SQLite table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS district_transaction (
    state TEXT,
    district TEXT,
    year INTEGER,
    quarter INTEGER,
    name TEXT,
    count INTEGER,
    amount REAL
)
""")

# ✅ Correct base path for district-level data
district_base = "pulse/data/map/transaction/hover/country/india/state/"

# Loop through states, years, and JSON files
for state in os.listdir(district_base):
    state_path = os.path.join(district_base, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if file.endswith(".json"):
                with open(os.path.join(year_path, file)) as f:
                    data = json.load(f)
                    try:
                        for txn in data['data']['hoverDataList']:
                            cursor.execute("""
                                INSERT INTO district_transaction
                                (state, district, year, quarter, name, count, amount)
                                VALUES (?, ?, ?, ?, ?, ?, ?)
                            """, (
                                state.replace("-", " ").title(),
                                txn['name'],  # district name
                                int(year),
                                int(file.strip(".json")),
                                txn['metric'][0]['type'],  # transaction type (e.g., TOTAL)
                                txn['metric'][0]['count'],
                                txn['metric'][0]['amount']
                            ))
                    except (KeyError, TypeError):
                        pass  # skip malformed or missing data
conn.commit()
print("✅ District-level data loaded successfully")


In [ ]:
# 6️⃣ Load data from SQLite into Pandas
df = pd.read_sql("SELECT * FROM aggregated_transaction", conn)
df.head()

In [ ]:
# 7️⃣ Basic Analysis: Top 10 States by Transaction Amount
top_states = df.groupby("state")["amount"].sum().sort_values(ascending=False).head(10)
print("Top 10 States by Transaction Amount:")
print(top_states)

plt.figure(figsize=(12,5))
top_states.plot(kind="bar", color="skyblue")
plt.title("Top 10 States by Transaction Amount")
plt.xlabel("State")
plt.ylabel("Total Amount (₹)")
plt.show()

In [ ]:
# 9️⃣ Save processed data to Google Drive
df.to_csv("/content/drive/MyDrive/phonepe_analysis.csv", index=False)
print("✅ Analysis CSV saved to Google Drive")

In [ ]:
# ✅ Working Dashboard for Google Colab (with Run button)

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import sqlite3

# --- Dropdown options ---
years = sorted(df['year'].unique())
quarters = sorted(df['quarter'].unique())
states = sorted(df['state'].unique())

# --- Widgets ---
year_dd = widgets.Dropdown(options=years, description="Year")
quarter_dd = widgets.Dropdown(options=quarters, description="Quarter")
state_dd = widgets.Dropdown(options=states, description="State")
run_btn = widgets.Button(description="Show Dashboard", button_style='success')
output = widgets.Output()

# --- Dashboard Function ---
def show_dashboard(b):
    with output:
        clear_output()

        year = year_dd.value
        quarter = quarter_dd.value
        state = state_dd.value

        # Filter data
        filtered = df[(df['year']==year) & (df['quarter']==quarter)]
        state_filtered = filtered[filtered['state']==state]

        total_txn = state_filtered['count'].sum()
        total_amt = state_filtered['amount'].sum()

        print(f"📊 Dashboard for {state} | Year: {year} | Quarter: {quarter}")
        print(f"Total Transactions: {total_txn:,}")
        print(f"Total Amount (₹): {total_amt/1e6:.2f} Million\n")

        # --- Bar chart ---
        if not state_filtered.empty:
            bar_df = state_filtered.groupby('name', as_index=False)['amount'].sum()
            fig1 = px.bar(bar_df, x='name', y='amount', color='name',
                          title=f"{state} — Transaction Amount by Payment Type",
                          text_auto='.2s')
            fig1.update_layout(xaxis_title="Payment Type", yaxis_title="Transaction Amount (₹)")
            fig1.show()

        # --- Top 5 Districts ---
        district_df = pd.read_sql(f"""
            SELECT * FROM district_transaction
            WHERE state='{state}' AND year={year} AND quarter={quarter}
        """, conn)

        if not district_df.empty:
            top_districts = (
                district_df.groupby('district', as_index=False)['amount'].sum()
                .sort_values(by='amount', ascending=False)
                .head(5)
            )
            fig2 = px.bar(top_districts, x='district', y='amount', color='amount',
                          title=f"Top 5 Districts in {state} by Transaction Amount",
                          text_auto='.2s')
            fig2.update_layout(xaxis_title="District", yaxis_title="Transaction Amount (₹)")
            fig2.show()
        else:
            print("⚠️ District-level data not available for this selection.\n")

        # --- India Map ---
        map_df = filtered.groupby('state', as_index=False)['amount'].sum()
        geojson_url = "https://raw.githubusercontent.com/geohacker/india/master/state/india_telengana.geojson"
        fig3 = px.choropleth(
            map_df,
            geojson=geojson_url,
            featureidkey="properties.st_nm",
            locations="state",
            color="amount",
            color_continuous_scale="Purples",
            title="India Transaction Amounts by State"
        )
        fig3.update_geos(fitbounds="locations", visible=False)
        fig3.show()

# --- Connect button ---
run_btn.on_click(show_dashboard)

# --- Display UI ---
display(widgets.VBox([year_dd, quarter_dd, state_dd, run_btn, output]))
